In [7]:
# 根据csv文件分析差分路径与差分的概率、密钥空间
# 注意修改prob
# ARX toolkit生成的csv文件要先打开再保存一次，使读到的dataframe第一列为0123...

prob = 50  # weight为35至prob的路径
num = 0 - prob - 1
num2 = num  
num3 = num
num4 = num# 用于计算概率的参数


import math
import pandas as pd
df1 = pd.read_csv(str(prob) + '.csv')
print(df1)

beginw = 0
for i in range(len(df1.index.values)-3):
    beginw += df1['w'][i]  # 用于计算开始时的概率
    
print("删除！或=：")
# 计算！=兼容性的轮数
alldelweightsum = 0  # 累加参数，以计算概率
rnd = 1  
while 1:
    sumw1 = 0 # ！权重
    sumw2 = 0 # = 权重
    w1 = ['sum_!',-1,] # 存放 ！权重的列表
    w2 = ['sum_=',-1,] # 存放 = 权重的列表
    minw = ['min(!_=)',-1,]# 存放 ！与 = 最小权重的列表
    for i in df1.columns.values[2:]: # i列名，不包括前两列
        for j in range(len(df1[i][:-3])): # j行数，不包括后3行
            if df1[i][j] == '!':
    #             print(j) ! 所在的那一行
    #             df1['w'][j] 对应路径的权重
                sumw1 += df1['w'][j]
            elif df1[i][j] == '=':
                sumw2 += df1['w'][j]
        w1.append(sumw1)
        w2.append(sumw2)
        minw.append(min(sumw1,sumw2))
        sumw1 = 0
        sumw2 = 0
    df1.iloc[-3] = w1
    df1.iloc[-2] = w2
    df1.iloc[-1] = minw  # 后三行更新计算后的权重
    print('\nround',rnd,',W : ',max(minw[2:])) # （每一列最小权重）的最大值
    if max(minw[2:]) == 0:  # != 已留下其中一种     
        break
    rnd += 1
    # print(minw.index(max(minw[2:]))) # （每一列最小权重）的最大值  所在列的index，包括前面id,w两列
    alldelweightsum += max(minw[2:])
    print('total\tW : ',alldelweightsum,'\n')
    colid = minw.index(max(minw[2:]))
    # print(df1.iloc[-2,colid]) # 所在列 = 权重
    # print(df1.iloc[-3,colid]) # 所在列 ！权重
    # print(df1.columns.values[2:][colid-2]) # 所在列名
    # print(df1[df1.columns.values[2:][colid-2]][:-3]) # 所在列的除了后三行的数据
    # print(len(df1[df1.columns.values[2:][colid-2]][:-3])) # 所在列的除了后三行的数据的长度
    dellist = [] # 待删除的行id
    if df1.iloc[-2,colid] == max(minw[2:]): # 是 = ,要删除有 = 的行
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-3])): # j 行数
    #         print(df1.iloc[j,colid])
            if df1.iloc[j,colid] == '=':
                dellist.append(j)
        df1.iloc[-2,colid] = 0 # 修改权重为0
    else:
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-3])): # j 行数
            if df1.iloc[j,colid] == '!':
                dellist.append(j)
        df1.iloc[-3,colid] = 0 # 修改权重为0
    df1.iloc[-1,colid] = 0 # 修改最小权重为0  
    df1 = df1.drop(dellist)  #删除行
    dic = dict(zip(df1.index.values.tolist(), range(len(df1['id']))))
    df1 = df1.rename(index=dic)
# ! = 分析完成
# print(df1)
print('over. total weight is: ',alldelweightsum)

# 计算不考虑2.5bit约束的概率
l1 = []
for i in bin(alldelweightsum)[2:]:
    l1.append(i)
l1.reverse()
summ = 0
for i in l1:
    num += 1
    if int(i) != 0:
        summ += pow(2, num)
print('至少的不兼容的概率(2为底，忽略2.5bit约束):',math.log(summ, 2))
#         print(math.log((pow(2,-30.39)-pow(2,-31)),2))
print('修正后概率(2为底，2^-30.39 - 至少的不兼容的概率):',math.log((pow(2,-30.39)-pow(2,math.log(summ, 2))),2)) 


# 计算含有多比特限制的列
print('含有多比特限制的列:')
for i in df1.columns.values[2:]: # i, 列名，不包括前两个
    multibit = set()
    for j in range(len(df1[i][:-3])): # j 行数
        if (not (pd.isnull(df1[i][j]))) and len(str(df1[i][j])) > 6:
            if j not in multibit:
                multibit.add(df1[i][j])
#                     break
    if not len(multibit)  == 0:
        print(i,multibit)        
df1.to_csv(str(prob) + '!=.csv')


print('\n\n2.5bit约束分析：')
print('41000082 =！或！=')
print('82000041 ==或！！，不能2个同时成立。')
print('430000c2 =！或！=或！！')
print('c2000043 =！或==或！！')
print('830000c1 ==或！！或！=')
print('c1000083 ==或=！或！=，不能4个同时成立')


temp = ['sum_41000082']
for i in range(101):
    temp.append(-1)
df1.loc[len(df1.index.values) + 1] = temp
temp[0] = 'sum_82000041'
df1.loc[len(df1.index.values) + 1] = temp
temp[0] = 'min(41000082_82000041)'
df1.loc[len(df1.index.values) + 1] = temp  # 添加3行

alldelweightsum2 = 0  # 累加参数，以计算概率


print("\n\n删除41000082或82000041，可能还会删除另外4个约束：")
rnd = 1  # 计算41000082、82000041兼容性的轮数
while 1:
    sumw1 = 0 # 41000082权重
    sumw2 = 0 # 82000041权重    
    sum430000c2 = 0
    sumc1000083 = 0
    sum830000c1 = 0
    sumc2000043 = 0  # 权重
    w1 = ['sum_41000082',-1,] # 存放 41000082权重的列表
    w2 = ['sum_82000041',-1,] # 存放 82000041权重的列表
    minw = ['min(41000082_82000041)',-1,]# 存放最小权重的列表
    w430000c2 = ['sum_430000c2',-1]
    wc1000083 = ['sum_c1000083',-1]
    w830000c1 = ['sum_830000c1',-1]
    wc2000043 = ['sum_c2000043',-1]  # 存放权重的列表
    for i in df1.columns.values[2:]: # i, 列名，不包括前两列
        for j in range(len(df1[i][:-6])): # j 行数
            if df1[i][j] == '41000082':
                sumw1 += df1['w'][j]
            elif df1[i][j] == '82000041':
                sumw2 += df1['w'][j]
            elif df1[i][j] == '430000c2':
                sum430000c2 += df1['w'][j]
            elif df1[i][j] == 'c1000083':
                sumc1000083 += df1['w'][j]
            elif df1[i][j] == '830000c1':
                sum830000c1 += df1['w'][j]
            elif df1[i][j] == 'c2000043':
                sumc2000043 += df1['w'][j]
        w1.append(sumw1)
        w2.append(sumw2)
        minw.append(min(sumw1,sumw2))
        w430000c2.append(sum430000c2)
        wc1000083.append(sumc1000083)
        w830000c1.append(sum830000c1)
        wc2000043.append(sumc2000043)
        sumw1 = 0
        sumw2 = 0
        sum430000c2 = 0
        sumc1000083 = 0
        sum830000c1 = 0
        sumc2000043 = 0
    df1.iloc[-3] = w1
    df1.iloc[-2] = w2
    df1.iloc[-1] = minw  # 更新最后三行计算后的权重
    if max(minw[2:]) == 0:  # 41000082、82000041已留下其中一种 
        print('\nround',rnd,'W : 0')
        break    
#     print(minw.index(max(minw[2:]))) # （每一列最小权重）的最大值  所在列的index，包括前面id,w两列
    colid = minw.index(max(minw[2:]))
    
#     !- or =- 的情况
    if (df1.iloc[-5,colid + 1] == 0) and (df1.iloc[-6,colid + 1] == 0):
        print('\nround',rnd,',W : ',max(minw[2:]))
        alldelweightsum2 += max(minw[2:])
        dellist = [] # 待删除的行id
        if df1.iloc[-2,colid] == max(minw[2:]): # 删除有82000041的行
            for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
        #         print(df1.iloc[j,colid])
                if df1.iloc[j,colid] == '82000041':
                    dellist.append(j)
            df1.iloc[-2,colid] = 0 # 修改权重为0
        else:  # # 删除有82000041的行
            for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
                if df1.iloc[j,colid] == '41000082':
                    dellist.append(j)
            df1.iloc[-3,colid] = 0 # 修改权重为0 
            
#     ==,保留82000041,删41000082和430000c2
    elif (int(df1.iloc[-5,colid]) > 0 and int(df1.iloc[-5,colid + 1]) > 0):        
        print('\nround',rnd,',W : ',df1.iloc[-2,colid] + w430000c2[colid])
        alldelweightsum2 += df1.iloc[-2,colid]
        alldelweightsum2 += w430000c2[colid]
        dellist = [] # 待删除的行id
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
            if df1.iloc[j,colid] == '41000082' or df1.iloc[j,colid] == '430000c2':
                dellist.append(j)
        df1.iloc[-2,colid] = 0 # 修改权重为0
        
#     !!，保留82000041,删41000082和c1000083      
    elif  (int(df1.iloc[-6,colid]) > 0 and int(df1.iloc[-6,colid + 1]) > 0):
        print('\nround',rnd,',W : ',df1.iloc[-2,colid] + wc1000083[colid])
        alldelweightsum2 += df1.iloc[-2,colid]
        alldelweightsum2 += wc1000083[colid]
        dellist = [] # 待删除的行id
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
            if df1.iloc[j,colid] == '41000082' or df1.iloc[j,colid] == 'c1000083':
                dellist.append(j)
        df1.iloc[-2,colid] = 0 # 修改权重为0 
        
#     =!,保留41000082,删82000041和830000c1
    elif (int(df1.iloc[-5,colid]) > 0 and int(df1.iloc[-6,colid + 1]) > 0):
        print('\nround',rnd,',W : ',df1.iloc[-3,colid] + w830000c1[colid])
        alldelweightsum2 += df1.iloc[-3,colid]
        alldelweightsum2 += w830000c1[colid]
        dellist = [] # 待删除的行id
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
            if df1.iloc[j,colid] == '82000041' or df1.iloc[j,colid] == '830000c1':
                dellist.append(j)
        df1.iloc[-3,colid] = 0 # 修改权重为0
        
#     !=,保留41000082,删82000041和c2000043
    else:
        print('\nround',rnd,',W : ',df1.iloc[-3,colid] + wc2000043[colid])
        alldelweightsum2 += df1.iloc[-3,colid]
        alldelweightsum2 += wc2000043[colid]
        dellist = [] # 待删除的行id
        for j in range(len(df1[df1.columns.values[2:][colid-2]][:-6])): # j 行数
            if df1.iloc[j,colid] == '82000041' or df1.iloc[j,colid] == 'c2000043':
                dellist.append(j)
        df1.iloc[-3,colid] = 0 # 修改权重为0    
        
    df1.iloc[-1,colid] = 0 # 修改最小权重为0    
    print('total\tW : ',alldelweightsum2,'\n')   
    df1 = df1.drop(dellist)  #删除行    
    dic = dict(zip(df1.index.values.tolist(), range(len(df1['id']))))
    df1 = df1.rename(index=dic)
    rnd += 1
# print(df1)


alldelweightsum = alldelweightsum + alldelweightsum2
print('over. total weight is: ',alldelweightsum)
print("\n\n处理未考虑到的1000082或82000041与！！、！=、=！、！！的情况：")
# 处理未考虑到的1000082或82000041与！！、！=、=！、！！的情况
alldelweightsum3 = 0
rnd = 1
sumw1 = 0 # 41000082权重
sumw2 = 0 # 82000041权重    
sum430000c2 = 0
sumc1000083 = 0
sum830000c1 = 0
sumc2000043 = 0  # 权重
k = 2
for i in df1.columns.values[2:-1]: # i, 列名，不包括前两列
    for j in range(len(df1[i][:-6])): # j 行数
        if df1[i][j] == '41000082':
            sumw1 += df1['w'][j]
        elif df1[i][j] == '82000041':
            sumw2 += df1['w'][j]
        elif df1[i][j] == '430000c2':
            sum430000c2 += df1['w'][j]
        elif df1[i][j] == 'c1000083':
            sumc1000083 += df1['w'][j]
        elif df1[i][j] == '830000c1':
            sum830000c1 += df1['w'][j]
        elif df1[i][j] == 'c2000043':
            sumc2000043 += df1['w'][j]
    dellist = [] # 待删除的行id
    if (int(df1.iloc[-5,k]) > 0) and (int(df1.iloc[-5,k+1]) > 0):  # ==,删除41000082、430000c2
        print('\nround',rnd,',W : ',sumw1 + sum430000c2)
        alldelweightsum3 += sumw1
        alldelweightsum3 += sum430000c2
        for j in range(len(df1.index.values) - 6):
            if df1[i][j] == '41000082' or df1[i][j] == '430000c2':
                dellist.append(j)
                
        df1.iloc[-3,k] = 0 # 修改权重为0
    elif (int(df1.iloc[-6,k]) > 0) and (int(df1.iloc[-6,k+1]) > 0):  # !!,删除41000082、c1000083
        print('\nround',rnd,',W : ',sumw1 + sumc1000083)
        alldelweightsum3 += sumw1
        alldelweightsum3 += sumc1000083
        for j in range(len(df1.index.values) - 6):
            if df1[i][j] == '41000082' or df1[i][j] == 'c1000083':
                dellist.append(j)
        df1.iloc[-3,k] = 0 # 修改权重为0
    elif (int(df1.iloc[-5,k]) > 0) and (int(df1.iloc[-6,k+1]) > 0):  # =!,删除82000041、830000c1
        print('\nround',rnd,',W : ',sumw2 + sum830000c1)
        alldelweightsum3 += sumw2
        alldelweightsum3 += sum830000c1
        for j in range(len(df1.index.values) - 6):
            if df1[i][j] == '82000041' or df1[i][j] == '830000c1':
                dellist.append(j)
        df1.iloc[-2,k] = 0 # 修改权重为0
    elif (int(df1.iloc[-6,k]) > 0) and (int(df1.iloc[-5,k+1]) > 0):  # !=,删除82000041、c2000043
        print('\nround',rnd,',W : ',sumw2 + sumc2000043)
        alldelweightsum3 += sumw2
        alldelweightsum3 += sumc2000043
        for j in range(len(df1.index.values) - 6):
            if df1[i][j] == '82000041' or df1[i][j] == 'c2000043':
                dellist.append(j)
        df1.iloc[-2,k] = 0 # 修改权重为0
    else:
        k += 1
        continue        
    sumw1 = 0
    sumw2 = 0
    sum430000c2 = 0
    sumc1000083 = 0
    sum830000c1 = 0
    sumc2000043 = 0       
    print('total\tW : ',alldelweightsum3,'\n')   
    df1 = df1.drop(dellist)  #删除行    
    dic = dict(zip(df1.index.values.tolist(), range(len(df1['id']))))
    df1 = df1.rename(index=dic)
    rnd += 1
    k += 1
# print(df1)
alldelweightsum += alldelweightsum3
print('over. total weight is: ',alldelweightsum)



# 添加4行，补充另外4种约束的情况。
df1.loc[len(df1.index.values) + 1] = w430000c2
df1.loc[len(df1.index.values) + 1] = wc1000083
df1.loc[len(df1.index.values) + 1] = w830000c1
df1.loc[len(df1.index.values) + 1] = wc2000043  


# 更新表
sumw1 = 0 # ！权重
sumw2 = 0 # =权重 
sum41000082 = 0
sum82000041 = 0
sum430000c2 = 0
sumc1000083 = 0
sum830000c1 = 0
sumc2000043 = 0  # 权重
w1 = ['sum_!',-1,] # 存放 ！权重的列表
w2 = ['sum_=',-1,] # 存放 = 权重的列表
min1 = ['min(!_=)',-1,]# 存放 ！与 = 最小权重的列表
w41000082 = ['sum_41000082',-1,] # 存放 41000082权重的列表
w82000041 = ['sum_82000041',-1,] # 存放 82000041权重的列表
min2 = ['min(41000082_82000041)',-1,]# 存放最小权重的列表
w430000c2 = ['sum_430000c2',-1]
wc1000083 = ['sum_c1000083',-1]
w830000c1 = ['sum_830000c1',-1]
wc2000043 = ['sum_c2000043',-1]  # 存放权重的列表

for i in df1.columns.values[2:]: # i, 列名，不包括前两列
    for j in range(len(df1[i][:-10])): # j 行数
        if df1[i][j] == '!':
            sumw1 += df1['w'][j]
        elif df1[i][j] == '=':
            sumw2 += df1['w'][j]
        elif df1[i][j] == '41000082':
            sum41000082 += df1['w'][j]
        elif df1[i][j] == '82000041':
            sum82000041 += df1['w'][j]
        elif df1[i][j] == '430000c2':
            sum430000c2 += df1['w'][j]
        elif df1[i][j] == 'c1000083':
            sumc1000083 += df1['w'][j]
        elif df1[i][j] == '830000c1':
            sum830000c1 += df1['w'][j]
        elif df1[i][j] == 'c2000043':
            sumc2000043 += df1['w'][j]
    w1.append(sumw1)
    w2.append(sumw2)
    min1.append(min(sumw1,sumw2))
    w41000082.append(sum41000082)
    w82000041.append(sum82000041)
    min2.append(min(sum41000082,sum82000041))
    w430000c2.append(sum430000c2)
    wc1000083.append(sumc1000083)
    w830000c1.append(sum830000c1)
    wc2000043.append(sumc2000043)
    sumw1 = 0 # ！权重
    sumw2 = 0 # =权重 
    sum41000082 = 0
    sum82000041 = 0
    sum430000c2 = 0
    sumc1000083 = 0
    sum830000c1 = 0
    sumc2000043 = 0  # 权重
df1.iloc[-10] = w1    
df1.iloc[-9] = w2
df1.iloc[-8] = min1
df1.iloc[-7] = w41000082  
df1.iloc[-6] = w82000041
df1.iloc[-5] = min2
df1.iloc[-4] = w430000c2  
df1.iloc[-3] = wc1000083
df1.iloc[-2] = w830000c1
df1.iloc[-1] = wc2000043  # 更新最后10行计算后的权重


print('删除430000c2、1000083、830000c1、c2000043其中一个：')
# 判断 430000c2、1000083、830000c1、c2000043 是否兼容
k = 2
m = ['430000c2','c1000083','830000c1','c2000043']
for i in df1.columns.values[2:]: # i, 列名，不包括前两列
    multibit = []
    t = 99999
    for j in range(-1,-5,-1): # 倒数
        if int(df1.iloc[j,k]) < t:
            t = int(df1.iloc[j,k])
            n = j
    if t > 0:  # 430000c2、1000083、830000c1、c2000043都存在，不兼容
        alldelweightsum += t
        print('430000c2、1000083、830000c1、c2000043 -> add W : ',t)
        dellist = []
        for l in range(len(df1.index.values) - 10): # j 行数
            if df1[i][l] == m[n]:
                dellist.append(l)                
        df1.iloc[n,k] = 0 # 修改权重为0 
        df1 = df1.drop(dellist)  #删除行    
        dic = dict(zip(df1.index.values.tolist(), range(len(df1['id']))))
        df1 = df1.rename(index=dic)
    k += 1



# 最后更新表
sumw1 = 0 # ！权重
sumw2 = 0 # =权重 
sum41000082 = 0
sum82000041 = 0
sum430000c2 = 0
sumc1000083 = 0
sum830000c1 = 0
sumc2000043 = 0  # 权重
w1 = ['sum_!',-1,] # 存放 ！权重的列表
w2 = ['sum_=',-1,] # 存放 = 权重的列表
min1 = ['min(!_=)',-1,]# 存放 ！与 = 最小权重的列表
w41000082 = ['sum_41000082',-1,] # 存放 41000082权重的列表
w82000041 = ['sum_82000041',-1,] # 存放 82000041权重的列表
min2 = ['min(41000082_82000041)',-1,]# 存放最小权重的列表
w430000c2 = ['sum_430000c2',-1]
wc1000083 = ['sum_c1000083',-1]
w830000c1 = ['sum_830000c1',-1]
wc2000043 = ['sum_c2000043',-1]  # 存放权重的列表

for i in df1.columns.values[2:]: # i, 列名，不包括前两列
    for j in range(len(df1[i][:-10])): # j 行数
        if df1[i][j] == '!':
    #             print(j) ! 所在的那一行
    #             df1['w'][j] 对应路径的权重
            sumw1 += df1['w'][j]
        elif df1[i][j] == '=':
            sumw2 += df1['w'][j]
        elif df1[i][j] == '41000082':
            sum41000082 += df1['w'][j]
        elif df1[i][j] == '82000041':
            sum82000041 += df1['w'][j]
        elif df1[i][j] == '430000c2':
            sum430000c2 += df1['w'][j]
        elif df1[i][j] == 'c1000083':
            sumc1000083 += df1['w'][j]
        elif df1[i][j] == '830000c1':
            sum830000c1 += df1['w'][j]
        elif df1[i][j] == 'c2000043':
            sumc2000043 += df1['w'][j]
    w1.append(sumw1)
    w2.append(sumw2)
    min1.append(min(sumw1,sumw2))
    w41000082.append(sum41000082)
    w82000041.append(sum82000041)
    min2.append(min(sum41000082,sum82000041))
    w430000c2.append(sum430000c2)
    wc1000083.append(sumc1000083)
    w830000c1.append(sum830000c1)
    wc2000043.append(sumc2000043)
    sumw1 = 0 # ！权重
    sumw2 = 0 # =权重 
    sum41000082 = 0
    sum82000041 = 0
    sum430000c2 = 0
    sumc1000083 = 0
    sum830000c1 = 0
    sumc2000043 = 0  # 权重
df1.iloc[-10] = w1    
df1.iloc[-9] = w2
df1.iloc[-8] = min1
df1.iloc[-7] = w41000082  
df1.iloc[-6] = w82000041
df1.iloc[-5] = min2
df1.iloc[-4] = w430000c2  
df1.iloc[-3] = wc1000083
df1.iloc[-2] = w830000c1
df1.iloc[-1] = wc2000043  # 更新最后10行计算后的权重


print('all over. total weight is: ',alldelweightsum)

# 计算开始时所有路径的的总概率
print('begin weight: ',beginw)
l1 = []
for i in bin(beginw)[2:]:
    l1.append(i)
l1.reverse()
summ = 0
for i in l1:
    num2 += 1
    if int(i) != 0:
        summ += pow(2, num2)
pbefore = math.log(summ, 2)
print('计算前的概率(2为底):',pbefore)


# 计算后概率
l1 = []
for i in bin(alldelweightsum)[2:]:
    l1.append(i)
l1.reverse()
summ = 0
for i in l1:
    num3 += 1
    if int(i) != 0:
        summ += pow(2, num3)
print('至少的不兼容的概率(2为底):',math.log(summ, 2))
#         print(math.log((pow(2,-30.39)-pow(2,-31)),2))
pafter1 = math.log((pow(2,pbefore)-pow(2,math.log(summ, 2))),2)
print('修正后概率(2为底，计算前的概率 - 至少的不兼容的概率):',pafter1)
pafter2 = math.log((pow(2,-30.39)-pow(2,math.log(summ, 2))),2)
print('修正后概率(2为底，2^-30.39 - 至少的不兼容的概率):',pafter2)

df1.to_csv(str(prob) + 'all.csv')


# 计算删除部分低概率路径后的弱密钥空间以及概率
threshold = 600

# 处理表
df1 = df1.drop([len(df1.index.values)-5,len(df1.index.values)-8])
df1 = df1[-8:]
dic = dict(zip(df1.index.values.tolist(), range(len(df1['id']))))
df1 = df1.rename(index=dic)
#  增加一行，计算所在列的总和
temp = ['all','-1']
for i in range(100):
    temp.append(-1)
df1.loc[len(df1.index.values)] = temp
applist = ['all','-1',0,0]
summ = 0
for i in df1.columns.values[4:-2]: # i, 列名，不包括前两列
    for j in range(0,8,1):
        summ += int(df1[i][j])
    applist.append(summ)
    summ = 0
applist.append(0)
applist.append(0)
df1.iloc[8] = applist  # 更新最后10行计算后的权重
# print(df1)

# 删除列 <600的约束
k = 3
for i in df1.columns.values[4:-2]:
    k+=1
    if (int(df1.iloc[8,k]) > 0) and (int(df1.iloc[8,k]) < threshold):
        alldelweightsum += df1.iloc[8,k]
        df1.iloc[8,k] = 0

        
# 计算弱密钥空间
m = ['41000082','82000041','430000c2','c1000083','830000c1','c2000043','']
eq = ['=!','==','=!','=-','==','=-','']
neq = ['!=','!!','!-','!!','!-','!=','']
print('弱密钥空间：')
print('k0:---- ---- ---- ----')
print('k1:---- ---- ---- ----',end = '')
k = 3
b = False
l = -1
m = 2
weakk = 0
for i in df1.columns.values[4:-2]: # i, 列名，不包括前两列
    l += 1
    k += 1
    if l % 16 == 0:
        print('\nk' + str(m) + ':',end = '')
        m += 1
    elif l % 4 == 0:
        print(' ',end = '')
    if b == True:
        b = False
        continue
    if (int(df1.iloc[1,k]) > 0) and (int(df1.iloc[8,k]) > 0): # =
        if (int(df1.iloc[1,k+1]) == 0) and (int(df1.iloc[0,k+1]) == 0):  # =-,然后看2.5bit
            for j in range(-7,-1,1):
                if (int(df1.iloc[j,k]) > 0):
                    if (int(df1.iloc[j,k]) > threshold):
                        print(eq[j],end = '')
                        weakk += 2
                        b = True
                        break
                    else:
                        alldelweightsum += int(df1.iloc[j,k])
            if b == False:
                print('=',end = '')
                weakk += 1
        else:
            print('=',end = '')
            weakk += 1
    elif (int(df1.iloc[0,k]) > 0) and (int(df1.iloc[8,k]) > 0): # !
        if (int(df1.iloc[1,k+1]) == 0) and (int(df1.iloc[0,k+1]) == 0):  # !-,然后看2.5bit
            for j in range(-7,-1,1):
                if (int(df1.iloc[j,k]) > 0):
                    if (int(df1.iloc[j,k]) > threshold):
                        print(neq[j],end = '')
                        weakk += 2
                        b = True
                        break
                    else:
                        alldelweightsum += int(df1.iloc[j,k])
            if b == False:
                print('!',end = '')
                weakk += 1
        else:
            print('!',end = '')
            weakk += 1
    else:
        print('-',end = '')
print('\nk8:---- ---- ---- ----')
print('k9:---- ---- ---- ----') 


print('all over. total weight is: ',alldelweightsum)
print('计算前的概率(2为底):',pbefore)
print('删除部分低概率路径后的概率：')

l1 = []
for i in bin(alldelweightsum)[2:]:
    l1.append(i)
l1.reverse()
summ = 0
for i in l1:
    num4 += 1
    if int(i) != 0:
        summ += pow(2, num4)
print('至少的不兼容的概率(2为底):',math.log(summ, 2))
pafter1 = math.log((pow(2,pbefore)-pow(2,math.log(summ, 2))),2)
print('修正后概率(2为底，计算前的概率 - 至少的不兼容的概率):',pafter1)
pafter2 = math.log((pow(2,-30.39)-pow(2,math.log(summ, 2))),2)
print('修正后概率(2为底，2^-30.39 - 至少的不兼容的概率):',pafter2)

e = 64 - weakk
# print(e)
print('2^',pafter1,' for 2^',e,'keys')
print('0 for others')

             id      w   k0   k1 k2_15 k2_14 k2_13 k2_12 k2_11 k2_10  ...  \
0          35_0  32768  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
1          36_0  16384  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
2          37_0   8192  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
3          37_1   8192  NaN  NaN     -     -     -     -     -     -  ...   
4          37_2   8192  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
...         ...    ...  ...  ...   ...   ...   ...   ...   ...   ...  ...   
25253  50_10104      1  NaN  NaN     -     -     -     -     -     -  ...   
25254  50_10105      1  NaN  NaN   NaN   NaN   NaN   NaN   NaN   NaN  ...   
25255     sum_!     -1 -1.0 -1.0    -1    -1    -1    -1    -1    -1  ...   
25256     sum_=     -1 -1.0 -1.0    -1    -1    -1    -1    -1    -1  ...   
25257  min(!_=)     -1 -1.0 -1.0    -1    -1    -1    -1    -1    -1  ...   

      k7_7 k7_6 k7_5 k7_4 k7_3 k7_2 k7_1 k7_0   k8   k9  
0      NaN  NaN  